In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dropout_value = 0.02
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.convblock1 = nn.Sequential(
        nn.Conv2d(in_channels=12, out_channels=24, kernel_size=(3, 3), padding=1, bias=False),
        nn.ReLU(),
        nn.GroupNorm(num_groups = 8, num_channels = 24),
        nn.Dropout(dropout_value)
    )

    def forward(self,x):


In [2]:
input = torch.randn(3, 12, 4, 4)
# Separate 6 channels into 3 groups
# m = nn.GroupNorm(3, 6)
# # Separate 6 channels into 6 groups (equivalent with InstanceNorm)
# m = nn.GroupNorm(6, 6)
# # Put all 6 channels into a single group (equivalent with LayerNorm)
# m = nn.GroupNorm(1, 6)
# # Activating the module
# output = m(input)

In [5]:
print(input)


tensor([[[[ 5.2927e-01, -1.5380e-01, -5.1500e-01,  ..., -1.7445e+00,
            2.7787e-02, -8.5819e-01],
          [ 2.1982e-01, -3.1195e-01,  5.4999e-01,  ...,  6.6422e-01,
            2.1052e+00, -4.7920e-01],
          [-9.5356e-01,  1.5345e+00, -3.0968e-01,  ..., -1.5057e+00,
           -8.8518e-01,  1.1308e-01],
          ...,
          [ 9.6281e-01,  4.1023e-01,  2.9983e-01,  ..., -4.4432e-01,
           -7.3384e-02,  8.1859e-01],
          [-3.9205e-01,  9.8208e-01, -1.6880e+00,  ..., -1.6607e-01,
            1.7354e-01,  1.0075e-01],
          [-3.4180e-01,  4.4284e-01, -7.2893e-01,  ...,  2.0354e-01,
            1.2418e+00,  5.1133e-01]],

         [[ 7.5720e-01, -6.0668e-01,  1.4459e+00,  ..., -6.3269e-01,
           -5.9287e-01,  1.0529e+00],
          [-1.6503e-01, -1.4493e+00,  2.7979e-01,  ..., -2.8445e-02,
           -2.4139e-01,  3.4096e-02],
          [-2.3455e+00,  3.9553e-01,  1.8845e-01,  ...,  6.8497e-02,
           -1.9678e+00, -4.7006e-01],
          ...,
     

In [6]:
def get_normalisation(norm = 'Batch',input_channels = 0,num_groups = 0,**kwargs):
    
    '''

    Parameters:
        norm : Can be either 'Group' or 'Layer' or 'Batch'
        input_channels : no. of channels expected as input
        num_groups : Only valid for Group Normalisation, specifies the no. of groups channels will be divided into
        kwargs : for Layer Norm pass in a tuple or list or torch.Size as shape = N,C,H,W 
    
    Output:
        Returns object of the specified Normalisation to be applied

    '''
    if norm  == 'Group':
        return nn.GroupNorm(num_groups,input_channels)
    
    elif norm == 'Layer':
        return nn.LayerNorm(kwargs['shape'])
    
    return nn.BatchNorm2d(input_channels)
    

In [10]:
input = torch.randn(3, 12, 4, 4)
g = get_normalisation('Layer',shape = input.size())
output = g(input)
output.shape

torch.Size([3, 12, 4, 4])

In [20]:
dropout_value = 0.02
class Net4(nn.Module):
  def __init__(self,norm,batch_size = 2):
    super(Net4, self).__init__()

    self.batch_size = batch_size
    self.norm = norm
  
    # Input Block  LAYER  1
    self.convblock1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=1, bias=False),
        nn.ReLU(),
        get_normalisation(self.norm,input_channels=10,num_groups=2,shape = (self.batch_size,10,28,28)),
        nn.Dropout(dropout_value)
        ) # output_size = 28 rf=3

    self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            get_normalisation(self.norm,input_channels=16,num_groups=4,shape = (self.batch_size,16,28,28)),
            nn.Dropout(dropout_value)
        ) # output_size = 28 rf=5
    

    #TRANSITION BLOCK 1
    self.pool1 = nn.MaxPool2d(2, 2) # output_size = 14 rf=6
    self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1,1), padding=0, bias=False),
            nn.ReLU(),
            get_normalisation(self.norm,input_channels=10,num_groups=2,shape = (self.batch_size,10,14,14)),
            nn.Dropout(dropout_value)
        ) # output_size = 28 rf=10

    #LAYER 2
    self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=14, kernel_size=(3, 3), padding=0, bias=False),
            get_normalisation(self.norm,input_channels=14,num_groups=2,shape = (self.batch_size,14,12,12)),
            nn.ReLU()
        )
    self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=14, out_channels=24, kernel_size=(3, 3), padding=0, bias=False),
            nn.ReLU(),
            get_normalisation(self.norm,input_channels=24,num_groups=4,shape = (self.batch_size,24,10,10)),
            nn.Dropout(dropout_value)
        ) # output_size = 10 rf=14


    #TRANSITION BLOCK 2
    self.pool2 = nn.MaxPool2d(2, 2) # output_size = 5 rf=15
    self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=24, out_channels=10, kernel_size=(1,1), padding=0, bias=False),
            nn.ReLU(),
            get_normalisation(self.norm,input_channels=10,num_groups=2,shape = (self.batch_size,10,5,5)),
            nn.Dropout(dropout_value)
        ) # output_size = 28 rf=5
    
    #LAYER 3
    self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=10, out_channels=15, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            get_normalisation(self.norm,input_channels=15,num_groups=3,shape = (self.batch_size,15,5,5))
        ) # output_size = 3 rf=18
    self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=15, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            get_normalisation(self.norm,input_channels=15,num_groups=3,shape = (self.batch_size,15,5,5))
        ) # output_size = 3 rf=18

    #GLOBAL AVG POOLINNG
    self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=5)
        )# output size = 1 rf=34

    self.fc1 = nn.Linear(in_features=15, out_features=10)

  def forward(self,x):
    x = self.convblock1(x)
    x = self.convblock2(x)

    x = self.pool1(x)
    x = self.convblock3(x)
    
    x = self.convblock4(x)
    x = self.convblock5(x)

    x = self.pool2(x)
    x = self.convblock6(x)
    
    x = self.convblock7(x)
    x = self.convblock8(x)

    x = self.gap(x)
    x = x.view(-1,15)
    x = self.fc1(x)
      
    x = x.view(-1, 10)
    return F.log_softmax(x, dim=-1)

In [13]:
!pip install torchsummary


In [15]:
from torchsummary import summary

In [17]:
model = Net4(norm = 'Group')
summary(model,input_size = (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
              ReLU-2           [-1, 10, 28, 28]               0
         GroupNorm-3           [-1, 10, 28, 28]              20
           Dropout-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,440
              ReLU-6           [-1, 16, 28, 28]               0
         GroupNorm-7           [-1, 16, 28, 28]              32
           Dropout-8           [-1, 16, 28, 28]               0
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10           [-1, 10, 14, 14]             160
             ReLU-11           [-1, 10, 14, 14]               0
        GroupNorm-12           [-1, 10, 14, 14]              20
          Dropout-13           [-1, 10, 14, 14]               0
           Conv2d-14           [-1, 14,

In [22]:
model = Net4(norm = 'Layer')
summary(model,input_size = (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
              ReLU-2           [-1, 10, 28, 28]               0
         LayerNorm-3           [-1, 10, 28, 28]          31,360
           Dropout-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,440
              ReLU-6           [-1, 16, 28, 28]               0
         LayerNorm-7           [-1, 16, 28, 28]          50,176
           Dropout-8           [-1, 16, 28, 28]               0
         MaxPool2d-9           [-1, 16, 14, 14]               0
           Conv2d-10           [-1, 10, 14, 14]             160
             ReLU-11           [-1, 10, 14, 14]               0
        LayerNorm-12           [-1, 10, 14, 14]           7,840
          Dropout-13           [-1, 10, 14, 14]               0
           Conv2d-14           [-1, 14,